In [1]:
import sys
sys.path.append('../..')

In [2]:
import torch

from src.tool import config
from src import dataset, dataloader, model, loss, metric, optimizer
from src.tool.registry import DATASET_REGISTRY, DATALOADER_REGISTRY, MODEL_REGISTRY, LOSS_REGISTRY, METRIC_REGISTRY, OPTIMIZER_REGISTRY

# torch settings
torch.manual_seed(0)

if torch.cuda.is_available():
    device = 'cuda'
if torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

device

'mps'

In [3]:
# load options
path = '../../experiment/20210615/'
opt = config.load_config(path)
opt

CfgNode({'data': CfgNode({'dataset': 'Pedar_Dataset_static2dynamic', 'dataloader': 'DataLoader', 'train_ratio': 0.8, 'batch_size': 32}), 'model': CfgNode({'name': 'MLP'}), 'train': CfgNode({'loss': 'MSELoss', 'metric': '', 'optimizer': 'SGD', 'lr': 0.1, 'epoch': 100})})

In [22]:
dict(opt.data)

{'dataset': 'Pedar_Dataset_static2dynamic',
 'dataloader': 'DataLoader',
 'train_ratio': 0.8,
 'batch_size': 32}

## Data loading

In [4]:
from torch.utils.data import random_split

dataset = DATASET_REGISTRY[opt.data.dataset](static_path='output/pedar_static.pkl', dynamic_path='output/pedar_dynamic.pkl')
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
len(train_dataset), len(test_dataset)

(240, 60)

In [5]:
train_dataloader = DATALOADER_REGISTRY[opt.data.dataloader](train_dataset, batch_size=32, shuffle=True)
test_dataloader = DATALOADER_REGISTRY[opt.data.dataloader](test_dataset, batch_size=32, shuffle=True)

## Training & testing scheme

In [6]:
from typing import Dict, List, Tuple

def train_step(
        model: torch.nn.Module, 
        dataloader: torch.utils.data.DataLoader, 
        loss_fn: torch.nn.Module, 
        optimizer: torch.optim.Optimizer,
        device: torch.device) -> Tuple[float, float]:
    # put model in train mode
    model.train()
    train_loss = 0

    for batch, (X, y) in enumerate(dataloader):
        # send data to device
        X, y = X.to(device), y.to(device)

        # forward pass
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # loss backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # return metric
    train_loss = train_loss / len(dataloader)
    return train_loss

In [7]:
def test_step(
        model: torch.nn.Module, 
        dataloader: torch.utils.data.DataLoader, 
        loss_fn: torch.nn.Module,
        device: torch.device) -> Tuple[float, float]:
    # put model in eval mode
    model.eval() 
    test_loss = 0

    # turn on inference context manager
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            # send data to target device
            X, y = X.to(device), y.to(device)

            # forward pass
            test_pred_logits = model(X)

            # loss calculation
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

    # return metric
    test_loss = test_loss / len(dataloader)
    return test_loss

In [8]:
from tqdm.auto import tqdm

def train(
        model: torch.nn.Module, 
        train_dataloader: torch.utils.data.DataLoader, 
        test_dataloader: torch.utils.data.DataLoader, 
        optimizer: torch.optim.Optimizer,
        loss_fn: torch.nn.Module,
        epochs: int,
        device: torch.device) -> Dict[str, List]:
    # create results dict
    results = {
        "train_loss": [],
        "test_loss": [],
    }

    for epoch in tqdm(range(epochs)):
        train_loss = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
            )
        
        test_loss = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device,
            )

        # logs
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f} | "
        )

        results["train_loss"].append(train_loss)
        results["test_loss"].append(test_loss)

    return results

## Launch training

In [21]:
model = MODEL_REGISTRY[opt.model.name](hidden_size=256).to(device)
loss = LOSS_REGISTRY[opt.train.loss]()
optimizer = OPTIMIZER_REGISTRY[opt.train.optimizer](params=model.parameters(), lr=opt.train.lr)

results = train(model, train_dataloader, test_dataloader, optimizer, loss, opt.train.epoch, device)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.1689 | test_loss: 0.1638 | 
Epoch: 2 | train_loss: 0.1581 | test_loss: 0.1532 | 
Epoch: 3 | train_loss: 0.1478 | test_loss: 0.1432 | 
Epoch: 4 | train_loss: 0.1380 | test_loss: 0.1340 | 
Epoch: 5 | train_loss: 0.1289 | test_loss: 0.1251 | 
Epoch: 6 | train_loss: 0.1206 | test_loss: 0.1170 | 
Epoch: 7 | train_loss: 0.1124 | test_loss: 0.1092 | 
Epoch: 8 | train_loss: 0.1049 | test_loss: 0.1020 | 
Epoch: 9 | train_loss: 0.0981 | test_loss: 0.0951 | 
Epoch: 10 | train_loss: 0.0913 | test_loss: 0.0889 | 
Epoch: 11 | train_loss: 0.0852 | test_loss: 0.0829 | 
Epoch: 12 | train_loss: 0.0796 | test_loss: 0.0775 | 
Epoch: 13 | train_loss: 0.0744 | test_loss: 0.0723 | 
Epoch: 14 | train_loss: 0.0694 | test_loss: 0.0675 | 
Epoch: 15 | train_loss: 0.0648 | test_loss: 0.0632 | 
Epoch: 16 | train_loss: 0.0607 | test_loss: 0.0591 | 
Epoch: 17 | train_loss: 0.0565 | test_loss: 0.0552 | 
Epoch: 18 | train_loss: 0.0531 | test_loss: 0.0517 | 
Epoch: 19 | train_loss: 0.0496 | test